In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory('train',
                                              target_size=(128, 128),
                                              batch_size=32,
                                              class_mode='binary')

test_data = test_datagen.flow_from_directory('test',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 477 images belonging to 2 classes.
Found 476 images belonging to 2 classes.


In [3]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [4]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(train_data,
          steps_per_epoch=train_data.samples // 32,
          epochs=10,
          validation_data=test_data,
          validation_steps=test_data.samples // 32)

Epoch 1/10


14/14 [==============================] - 28s 2s/step - loss: 0.5137 - accuracy: 0.7955 - val_loss: 0.5126 - val_accuracy: 0.7991
Epoch 2/10
14/14 [==============================] - 12s 815ms/step - loss: 0.4900 - accuracy: 0.8000 - val_loss: 0.4487 - val_accuracy: 0.7991
Epoch 3/10
14/14 [==============================] - 14s 1s/step - loss: 0.4463 - accuracy: 0.8180 - val_loss: 0.4335 - val_accuracy: 0.8192
Epoch 4/10
14/14 [==============================] - 20s 1s/step - loss: 0.4176 - accuracy: 0.8427 - val_loss: 0.4024 - val_accuracy: 0.8504
Epoch 5/10
14/14 [==============================] - 11s 812ms/step - loss: 0.4079 - accuracy: 0.8594 - val_loss: 0.3875 - val_accuracy: 0.8504
Epoch 6/10
14/14 [==============================] - 11s 811ms/step - loss: 0.3857 - accuracy: 0.8607 - val_loss: 0.3650 - val_accuracy: 0.8638
Epoch 7/10
14/14 [==============================] - 10s 754ms/step - loss: 0.3768 - accuracy: 0.8539 - val_loss: 0.3636 - val_accuracy: 0.8527
Epoch 

In [6]:
loss, accuracy = model.evaluate(test_data)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

15/15 [==============================] - 4s 284ms/step - loss: 0.3327 - accuracy: 0.8655
Test loss: 0.3327
Test accuracy: 0.8655


In [7]:
model.save('honeybees.h5')

c:\Users\91768\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
from tensorflow.keras.models import load_model
model = load_model('honeybees.h5')
def preprocess_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = img / 255.0 
    img = np.expand_dims(img, axis=0) 
    return img
def predict_image(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)

    if prediction > 0.5:
        return 'healthy'
    else:
        return 'effected'

image_path = 'train/hb/3.jpg'
result = predict_image(image_path)
print(f'The image is a {result}')
image_path


1/1 [==============================] - 0s 117ms/step
The image is a healthy


'train/hb/3.jpg'